# Login to Docker Registry

In [ ]:
# Authenticate with Internal Docker Registry (hosted in Google Cloud, but can be anywhere)
!gcloud auth configure-docker --quiet

# Create a /tmp directory (needed in next steps)
!mkdir -p /tmp

# IGNORE WARNINGs BELOW!!

# Install Dependencies (Used by Fairing)

In [ ]:
!pip install fairing

In [ ]:
"""Trains and Evaluates the MNIST network using a feed dictionary."""
import argparse
import os
import sys
import time
import random
import logging

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

import fairing
fairing.config.set_builder(name='append')

INPUT_DATA_DIR = '/tmp/tensorflow/mnist/input_data/'
MAX_STEPS = 2000
BATCH_SIZE = 100
LEARNING_RATE = 0.3
HIDDEN_1 = 128
HIDDEN_2 = 32

# HACK: Ideally we would want to have a unique subpath for each instance of the job, but since we can't
# we are instead appending HOSTNAME to the logdir
LOG_DIR = os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                       'tensorflow/mnist/logs/fully_connected_feed/', os.getenv('HOSTNAME', ''))
MODEL_DIR = os.path.join(LOG_DIR, 'model.ckpt')

def train():
    data_sets = input_data.read_data_sets(INPUT_DATA_DIR)
    images_placeholder = tf.placeholder(
        tf.float32, shape=(BATCH_SIZE, mnist.IMAGE_PIXELS))
    labels_placeholder = tf.placeholder(tf.int32, shape=(BATCH_SIZE))

    logits = mnist.inference(images_placeholder,
                             HIDDEN_1,
                             HIDDEN_2)

    loss = mnist.loss(logits, labels_placeholder)
    train_op = mnist.training(loss, LEARNING_RATE)
    summary = tf.summary.merge_all()
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    sess = tf.Session()
    summary_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    sess.run(init)

    data_set = data_sets.train
    for step in xrange(MAX_STEPS):
        images_feed, labels_feed = data_set.next_batch(BATCH_SIZE, False)
        feed_dict = {
            images_placeholder: images_feed,
            labels_placeholder: labels_feed,
        }

        _, loss_value = sess.run([train_op, loss],
                                 feed_dict=feed_dict)
        if step % 100 == 0:
            print("At step {}, loss = {}".format(step, loss_value))
            summary_str = sess.run(summary, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()

# Launch Training Job on Kubernetes Cluster

In [ ]:
import fairing

DOCKER_REGISTRY = 'gcr.io/pipelineai2/fairing-jobs'

fairing.config.set_builder('append',  
                           registry=DOCKER_REGISTRY, 
                           push=True)
fairing.config.set_deployer('job')

In [ ]:
import fairing

fairing_train = fairing.config.fn(train)
fairing_train()